In [1]:
import numpy as np

In [77]:
def Input0(arr):
    new_arr=np.zeros((len(arr)),dtype=int)
    new_arr[1:]=arr[:-1]
    return new_arr

def Input1(arr):
    new_arr=np.ones((len(arr)),dtype=int)
    new_arr[1:]=arr[:-1]
    return new_arr

In [103]:
def decimalToBinary(digit):
    if digit == 0:
        binary = np.array([0,0,0])
    elif digit == 1:
        binary = np.array([0,0,1])
    elif digit == 2:
        binary = np.array([0,1,0])
    elif digit == 3:
        binary = np.array([0,1,1])
    elif digit == 4:
        binary = np.array([1,0,0])
    elif digit == 5:
        binary = np.array([1,0,1])
    elif digit == 6:
        binary = np.array([1,1,0])
    elif digit == 7:
        binary =np.array([1,1,1])
    else:
        print("Invalid Octal Digit")
    return binary

In [104]:
def octalToBinary(Octal):
    # octal = 13
    # binary = 000 000
    numOfDigits=len(str(abs(Octal)))
    polynomial=np.zeros((3*numOfDigits),dtype=int)

    # 2-digit case
    if numOfDigits == 2:
        b1=Octal//10
        b0=Octal%10
        polynomial[:3]=decimalToBinary(b1)
        polynomial[3:]=decimalToBinary(b0)
    # 3-digit case
    elif numOfDigits == 3:
        b2=Octal//100
        b1=(Octal%100)//10
        b0=Octal%10
        polynomial[:3]=decimalToBinary(b2)
        polynomial[3:6]=decimalToBinary(b1)
        polynomial[6:]=decimalToBinary(b0)
    # 4-digit case
    elif numOfDigits == 4:
        b3=Octal//1000
        b2=(Octal%1000)//100
        b1=(Octal%100)//10
        b0=Octal%10
        polynomial[:3]=decimalToBinary(b3)
        polynomial[3:6]=decimalToBinary(b2)
        polynomial[6:9]=decimalToBinary(b1)
        polynomial[9:]=decimalToBinary(b0)
    # 5-digit case
    elif numOfDigits == 5:
        b4=Octal//10000
        b3=(Octal%10000)//1000
        b2=(Octal%1000)//100
        b1=(Octal%100)//10
        b0=Octal%10
        polynomial[:3]=decimalToBinary(b4)
        polynomial[3:6]=decimalToBinary(b3)
        polynomial[6:9]=decimalToBinary(b2)
        polynomial[9:12]=decimalToBinary(b1)
        polynomial[12:]=decimalToBinary(b0)
    return polynomial

In [105]:
def binary_addition(a,b):
    if (a+b)%2==1:
        return 1
    elif (a+b)%2==0:
        return 0
    else:
        return "Invalid Input"

In [ ]:
class poly2trellis:
    def __init__(self,k,n,v,polynomial):
        self.k_=k
        self.n_=n
        self.v_=v
        self.p0_octal=polynomial[0]
        self.p1_octal=polynomial[1]
        self.numInputSymbols=2**self.k_
        self.numOutputSymbols=2**self.n_
        self.numStates=2**self.v_
        self.nextStates=self.computeNextStates()
        self.outputs=self.computeOutputs()

    def computeNextStates(self):
        state_matrix=np.zeros((self.numStates,self.numInputSymbols),dtype=int)
        for i in range(1,self.numStates+1):

            # initialize state: 000 (1x3)
            current_state=np.zeros((self.v_),dtype=int)
            state_in_binary=np.array([int(char) for char in bin(i-1)[2:]])
            start_index=len(current_state)-len(state_in_binary)
            current_state[start_index:]=state_in_binary.reshape(1,len(state_in_binary))    

            # input = 0
            next_state_in_binary=''.join(map(str,Input0(current_state)))
            next_state=int(next_state_in_binary,2)           
            state_matrix[i-1][0]=next_state

            # input = 1
            next_state_in_binary=''.join(map(str,Input1(current_state)))
            next_state=int(next_state_in_binary,2)           
            state_matrix[i-1][1]=next_state

        return state_matrix
    
    def computeOutputs(self):
        # p0=13 p1=17
        p0_binary=octalToBinary(self.p0_octal) # [0 0 1 0 1 1]
        p1_binary=octalToBinary(self.p1_octal) # [0 0 1 1 1 1]

        output_matrix=np.zeros((self.numStates,self.numInputSymbols), dtype=int)
        for i in range(1,self.numStates+1):
            # initialize state: 000 (1x3)
            current_state=np.zeros((self.v_),dtype=int)
            state_in_binary=np.array([int(char) for char in bin(i-1)[2:]])
            start_index=len(current_state)-len(state_in_binary)
            current_state[start_index:]=state_in_binary.reshape(1,len(state_in_binary))  


            output_in_binary=np.zeros(2, dtype=int)
            # input: 0
            u0=0
            # output: [c0 c1]
            c0=u0 
            c1=u0
            for j in range((-1)*self.v_,0):
                c0=binary_addition(c0,current_state[j+self.v_]*p0_binary[j])
                c1=binary_addition(c1,current_state[j+self.v_]*p1_binary[j])
            output_in_binary[0]=c0
            output_in_binary[1]=c1
            output_matrix[i-1][0]=int(''.join(map(str,output_in_binary)),2)

            output_in_binary=np.zeros(2,dtype=int)

            # input: 1
            u0=1
            # output: [c0 c1]
            c0=u0 
            c1=u0
            for j in range((-1)*self.v_,0):
                c0=binary_addition(c0,current_state[j+self.v_]*p0_binary[j])
                c1=binary_addition(c1,current_state[j+self.v_]*p1_binary[j])
            output_in_binary[0]=c0
            output_in_binary[1]=c1
            output_matrix[i-1][1]=int(''.join(map(str,output_in_binary)),2)
            
        return output_matrix



    